In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
### Célula para conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('Censo IBGE 2022/Compatibilização'):
    os.chdir('/content/drive/Shareddrives/SIG LabCidade/projetos/Censo IBGE 2022/Compatibilização')

Mounted at /content/drive


In [3]:
nome_compat = '2000-2010-2022'
nome_A = '2000'
nome_B = '2010'
nome_C = '2022'

## Importar dados de compatibilidade

In [4]:
# Importar perímetros compatíveis
gdf_perim = gpd.read_file(f'{nome_compat}/perimetros_compativeis.gpkg')
gdf_perim = gdf_perim.to_crs("EPSG:31983")

# Importar matrizes de compatibilidade
matriz_A = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_A}.csv',
                       sep='\t',
                       dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
matriz_B = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_B}.csv',
                       sep='\t',
                       dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})
matriz_C = pd.read_csv(f'{nome_compat}/matriz_compat_{nome_C}.csv',
                       sep='\t',
                       dtype={'CD_GEOCODI':str, 'CD_PERIMETRO':str})

In [5]:
gdf_perim['CD_MUN'] = gdf_perim['CD_PERIMETRO'].apply(lambda x: x[:7])
gdf_perim = gdf_perim.query('CD_MUN in @muns')

matriz_A['CD_MUN'] = matriz_A['CD_PERIMETRO'].apply(lambda x: x[:7])
matriz_A = matriz_A.query('CD_MUN in @muns')

matriz_B['CD_MUN'] = matriz_B['CD_PERIMETRO'].apply(lambda x: x[:7])
matriz_B = matriz_B.query('CD_MUN in @muns')

matriz_C['CD_MUN'] = matriz_C['CD_PERIMETRO'].apply(lambda x: x[:7])
matriz_C = matriz_C.query('CD_MUN in @muns')

## Importar dados IBGE

Dados comuns entre censos:

| Código | Variável | Localização em A | Localização em B | Localização em C |
|--------|----------|------------------|------------------|------------------|
| pop_tot | População total | Morador_SP_V0237 | Domicilio02_SP_V001 | v0001 |
| pop_dom_po | População em domicílios particulares | Morador_SP_V0238 | Domicilio02_SP_V002 (apenas permanentes) | v0005b = v0005 * v0007 |
| dom_tot | Domicílios totais | Domicilio_SP_V0001 | Domicilio01_SP_V001 | v0002 |
| dom_po | Domicílios particulares ocupados | Domicilio_SP_V0002 | Domicilio01_SP_V002 + DomicilioRenda_SP_V001 | v0007 |

In [28]:
# Dados malha A
loc_variaveis = [
    {
    'Domicilio_SP1': ['V0001', 'V0002', 'V0003', 'V0004'],
    'Domicilio_SP2': ['V0001', 'V0002', 'V0003', 'V0004'],},
     {
    'Morador_SP1': ['V0237', 'V0238', 'V0239'],
    'Morador_SP2': ['V0237', 'V0238', 'V0239'],},
]

data_A = matriz_A[['CD_GEOCODI']].dropna()
for group in loc_variaveis:
    df_group = pd.DataFrame()
    for file, variaveis in group.items():
        df_temp = pd.read_excel(f'../../../dados/IBGE 2000 Agregados por setores censitários/{file}.XLS',
                                usecols=['Cod_setor']+variaveis,
                                dtype={'Cod_setor':str})
        for v in variaveis:
            df_temp[v] = df_temp[v].replace('X',0).astype(float)
        df_temp = df_temp.rename(columns={k:f'{file[:-1]}_{k}' for k in variaveis})
        df_temp = df_temp.rename(columns={'Cod_setor':'CD_GEOCODI'})
        df_group = pd.concat([df_group, df_temp])
    data_A = data_A.merge(df_group, on='CD_GEOCODI', how='left')

### Mesclagem
data_A = data_A.merge(matriz_A, on='CD_GEOCODI', how='left')
data_A = data_A.query('CD_MUN in @muns')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [29]:
# Dados malha B
loc_variaveis = [
    {
    'Domicilio01_SP1': ['V001', 'V002'],
    'Domicilio01_SP2': ['V001', 'V002'],},
     {
    'Domicilio02_SP1': ['V001', 'V002'],
    'Domicilio02_SP2': ['V001', 'V002'],},
     {
    'DomicilioRenda_SP1': ['V001'],
    'DomicilioRenda_SP2': ['V001'],}
]
data_B = matriz_B[['CD_GEOCODI']].dropna()
for group in loc_variaveis:
    df_group = pd.DataFrame()
    for file, variaveis in group.items():
        df_temp = pd.read_csv(f'../../../dados/IBGE 2010 Agregados por setores censitários/{file}.csv',
                            sep=';',
                            usecols=['Cod_setor']+variaveis,
                            dtype={'Cod_setor':str})
        for v in variaveis:
            df_temp[v] = df_temp[v].replace('X',0).astype(float)
        df_temp = df_temp.rename(columns={k:f'{file[:-1]}_{k}' for k in variaveis})
        df_temp = df_temp.rename(columns={'Cod_setor':'CD_GEOCODI'})
        df_group = pd.concat([df_group, df_temp])
    data_B = data_B.merge(df_group, on='CD_GEOCODI', how='left')

### Mesclagem
data_B = data_B.merge(matriz_B, on='CD_GEOCODI', how='left')
data_B = data_B.query('CD_MUN in @muns')

In [23]:
# Dados malha C
data_C = pd.read_csv('../../../dados/IBGE 2022 Agregados preliminares por setores censitários/Agregados_preliminares_por_setores_censitarios_SP.csv',
            sep=';',
            dtype={'CD_SETOR':str})
data_C = data_C[['CD_SETOR', 'v0001', 'v0002', 'v0003', 'v0004', 'v0005', 'v0006', 'v0007']]
data_C = data_C.rename(columns={'CD_SETOR':'CD_GEOCODI'})
data_C['CD_GEOCODI'] = data_C['CD_GEOCODI'].apply(lambda x: x.replace('P',''))

### Mesclagem
data_C = data_C.merge(matriz_C, on='CD_GEOCODI', how='left')
data_C = data_C.query('CD_MUN in @muns')

## Reagregação

In [33]:
# Malha A
### Agregação
aggfuncs = {k:'sum' for k in data_A.columns if k not in ['CD_PERIMETRO', 'CD_GEOCODI']}
agg_A = data_A.pivot_table(index='CD_PERIMETRO', values=aggfuncs.keys(), aggfunc=aggfuncs).reset_index()

### Renomear variáveis
agg_A = agg_A.rename(columns={k:f'{nome_A}_{k}' for k in list(agg_A.columns) if k != 'CD_PERIMETRO'})

In [34]:
# Malha B
### Agregação
aggfuncs = {k:'sum' for k in data_B.columns if k not in ['CD_PERIMETRO', 'CD_GEOCODI']}
agg_B = data_B.pivot_table(index='CD_PERIMETRO', values=aggfuncs.keys(), aggfunc=aggfuncs).reset_index()

### Calcular dom_po
agg_B['dom_po'] = agg_B['Domicilio01_SP_V002'] + agg_B['DomicilioRenda_SP_V001']

### Renomear variáveis
agg_B = agg_B.rename(columns={k:f'{nome_B}_{k}' for k in list(agg_B.columns) if k != 'CD_PERIMETRO'})

In [35]:
# Malha C
### Moradores em Domicílios particulares ocupados
data_C['v0005b'] = data_C.apply(lambda x: int(round(x['v0005']*x['v0007'])), axis=1)

### Imputados de real para inteiro
data_C['v0006b'] = data_C.apply(lambda x: int(round(x['v0007']*(x['v0006']/100),0)), axis=1)

### Agregação
aggfuncs = {
    'v0001': 'sum',
    'v0002': 'sum',
    'v0003': 'sum',
    'v0004': 'sum',
    'v0005b': 'sum',
    'v0006b': 'sum',
    'v0007': 'sum'
}
agg_C = data_C.pivot_table(index='CD_PERIMETRO', values=aggfuncs.keys(), aggfunc=aggfuncs).reset_index()

### Recalcular variáveis compostas
agg_C['v0005'] = agg_C['v0005b']/agg_C['v0007']
agg_C['v0006'] = agg_C['v0006b']*100/agg_C['v0007']

### Renomear variáveis
agg_C = agg_C.rename(columns={k:f'{nome_C}_{k}' for k in list(agg_C.columns) if k != 'CD_PERIMETRO'})

In [36]:
agg = agg_C.merge(agg_B, on='CD_PERIMETRO', how='inner')
agg = agg.merge(agg_A, on='CD_PERIMETRO', how='inner')

## Renomeações

In [38]:
new_names = {'2022_v0001': 'pop_tot_2022',
             '2022_v0002': 'dom_tot_2022',
             '2022_v0003': 'dom_par_2022',
             '2022_v0004': 'dom_col_2022',
             '2022_v0005b': 'pop_dom_po_2022',
             '2022_v0006b': 'dom_po_imput_2022',
             '2022_v0007': 'dom_po_2022',
             '2022_v0005': 'med_pop_dom_po_2022',
             '2022_v0006': 'pct_dom_po_imput_2022',
             '2010_Domicilio01_SP_V001': 'dom_tot_2010',
             '2010_Domicilio01_SP_V002': 'dom_pp_2010',
             '2010_Domicilio02_SP_V001': 'pop_tot_2010',
             '2010_Domicilio02_SP_V002': 'pop_dom_pp_2010',
             '2010_DomicilioRenda_SP_V001': 'dom_pi_2010',
             '2010_dom_po': 'dom_po_2010',
             '2000_Domicilio_SP_V0001': 'dom_tot_2000',
             '2000_Domicilio_SP_V0002': 'dom_po_2000',
             '2000_Domicilio_SP_V0003': 'dom_pp_2000',
             '2000_Domicilio_SP_V0004': 'dom_pi_2000',
             '2000_Morador_SP_V0237': 'pop_tot_2000',
             '2000_Morador_SP_V0238': 'pop_dom_po_2000',
             '2000_Morador_SP_V0239': 'pop_dom_pp_2000',}

agg = agg.rename(columns=new_names)
agg = agg[sorted(agg.columns)]

## Cálculos de densidade

In [39]:
gdf_perim_agg = gdf_perim.merge(agg, on='CD_PERIMETRO', how='left')[[i for i in agg.columns]+['geometry']]

# Densidade populacional
for year in ['2000', '2010', '2022']:
    gdf_perim_agg[f'dens_pop_tot_ha_{year}'] = gdf_perim_agg[f'pop_tot_{year}']/(gdf_perim_agg['geometry'].area/10000)

    # Densidade domiciliar (total)
    gdf_perim_agg[f'dens_dom_tot_ha_{year}'] = gdf_perim_agg[f'dom_tot_{year}']/(gdf_perim_agg['geometry'].area/10000)

    # Densidade domiciliar (ocupados)
    gdf_perim_agg[f'dens_dom_po_ha_{year}'] = gdf_perim_agg[f'dom_po_{year}']/(gdf_perim_agg['geometry'].area/10000)

    # Média de moradores por domicílio
    if year != '2022':
        gdf_perim_agg[f'med_pop_dom_pp_{year}'] = gdf_perim_agg[f'pop_dom_pp_{year}']/gdf_perim_agg[f'dom_pp_{year}']

## Exportação

In [ ]:
agg.to_csv(f'{nome_compat}/agg_perimetros_{nome_A}-{nome_B}-{nome_C}.csv', sep='\t', index=False, decimal=',')

In [ ]:
gdf_perim_agg.to_file(f'{nome_compat}/perimetros_compativeis_agg.gpkg',
                      layer=f'{nome_A}-{nome_B}-{nome_C}',
                      driver='GPKG')